Carga de Librerias

In [1]:
import os
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import pickle
import urllib.request
import kagglehub
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #Graficar
import seaborn as sns
from collections import Counter
import math
import logging

c:\Users\eliana\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Descarga de Datos

In [2]:
# Descargar dataset con kagglehub (lo guarda en cache interna)
path = kagglehub.dataset_download("ranjeetjain3/seaborn-tips-dataset")

# Copiar el dataset descargado a la ruta que quieras
destino = "MLOPS_Project/data"
shutil.copytree(path, destino, dirs_exist_ok=True)

print("Dataset guardado en:", destino)

Dataset guardado en: MLOPS_Project/data


Lectura del Set de Datos

In [3]:
datos = pd.read_csv("MLOPS_Project/data/tips.csv")
datos.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [29]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [30]:
datos.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


Defino las variables a modelar

In [32]:
#Obtengo los Ln
continuas=datos[["total_bill","tip","size"]]
continuas_L = pd.DataFrame()
for col in continuas.columns:
    col_data = continuas[col]
    min_val = col_data.min()

    if (col_data < 0).any():
        desplazamiento = abs(min_val) + 1
        continuas_L[f"{col}_L"] = np.log(col_data + desplazamiento)

    elif (col_data == 0).any():
        continuas_L[f"{col}_L"] = np.log(col_data.replace(0, 1))
    else:
        continuas_L[f"{col}_L"] = np.log(col_data)

Definicion de Train y Test

In [33]:
np.random.seed(101)
sample = np.random.choice(datos.index, size=int(0.8 * len(datos)), replace=False)
train = datos.loc[sample]
test = datos.drop(sample)
display(train.head())
display(test.head())

,total_bill,tip,sex,smoker,day,time,size
151,13.13,2.00,Male,No,Sun,Dinner,2
34,17.78,3.27,Male,No,Sat,Dinner,2
109,14.31,4.00,Female,Yes,Sat,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
114,25.71,4.00,Female,No,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size
5,25.29,4.71,Male,No,Sun,Dinner,4
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
11,35.26,5.00,Female,No,Sun,Dinner,4
19,20.65,3.35,Male,No,Sat,Dinner,3


In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

df_train = df_train.copy()
df_val   = df_val.copy()
df_train["categorical_dict"] = df_train[categorical].to_dict(orient="records")
df_val["categorical_dict"]   = df_val[categorical].to_dict(orient="records")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", DictVectorizer(), "categorical_dict"),
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median")),
                          ("scaler", StandardScaler())]), numerical),
    ]
)

X_train = preprocessor.fit_transform(df_train)
y_train = df_train["duration"].values
X_val   = preprocessor.transform(df_val)
y_val   = df_val["duration"].values